In [1]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
import os.path as op
import os

inlist = lambda x, y: [x_ in y for x_ in x]

### Organize Dset100x1x1x20

In [2]:
df = pd.read_hdf('../data/impactofinstability_connect+feature_dset100x1x1x20_both.h5')
df = df.reset_index()

# Very incomplete sub: A00059344
inc = ['A00059344']
print(len(df))
for __ in inc:
    df = df[df.subject != __]
print(len(df))

# Broken graph subs
N_rois = 83
dfl = [row
       for idx, row in df.iterrows()
       if row['graph'].shape == (N_rois, N_rois)]
df = pd.DataFrame(dfl)
print(len(df))

# Drop pipelines and perturbation methods not of interest
df = df[df["pipeline"] == "prob"]
print(len(df))
df = df[df["instrumentation"] == "Data"]
print(len(df))
df.reset_index(drop=True, inplace=True)

# Drop some columns (will re-add relevant ones later)
df.drop(labels=['bmi', 'pulse', 'age', 'sex', 'index',
                'pipeline', 'instrumentation'], axis=1, inplace=True)

subs = list(set(df.subject))

8353
8309
8225
4111
2057


In [3]:
# Load DF
df_pheno = pd.read_csv('../data/phenotypic_w_kg.csv')

In [4]:
for cov in ['age', 'sex', 'bmi', 'cholesterol', 'rel_vo2max']:
    df[cov] = np.nan

for sub in subs:
    for cov in ['age', 'sex', 'bmi', 'cholesterol', 'rel_vo2max']:
        df.loc[df['subject'] == sub, cov] = df_pheno[df_pheno['id'] == sub][cov].values[0]

subs = list(df['subject'].unique())
for idx, row in df.iterrows():
    # Replace subids with unique numbers
    loc = subs.index(row.subject)
    row['subject'] = loc
    del loc
    
    row['age'] = int(float(row['age']) > 18)
    row['sex'] = int(row['sex']) - 1 # After this: M=0, F=1
    row['bmi'] = int(float(row['bmi']) > 25)
    row['rel_vo2max'] = int(float(row['rel_vo2max']) > (55 - (row['sex'])*10))
    row['cholesterol'] = int(float(row['cholesterol']) > 200)
    df.iloc[idx] = row
    del row

In [5]:
subdf = df.query('simulation == "ref"')

print(subdf['cholesterol'].value_counts())
print(subdf['age'].value_counts())
print(subdf['bmi'].value_counts())
print(subdf['rel_vo2max'].value_counts())
print(subdf['sex'].value_counts())

0.0    77
1.0    20
Name: cholesterol, dtype: int64
1.0    66
0.0    31
Name: age, dtype: int64
1.0    49
0.0    48
Name: bmi, dtype: int64
1.0    50
0.0    47
Name: rel_vo2max, dtype: int64
1.0    58
0.0    39
Name: sex, dtype: int64


In [6]:
df.to_hdf('../data/aggregation_connect+feature+demo_dset100x1x1x20.h5', 'data')

/Users/greg/code/env/agg/lib/python3.7/site-packages/pandas/core/generic.py:2449: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block2_values] [items->Index(['subject', 'simulation', 'graph', 'degree', 'weight', 'ccoeff',
       'betweenness', 'plength'],
      dtype='object')]

  encoding=encoding,


### Organize Dset25x2x2x20

In [7]:
df25 = pd.read_hdf('../data/impactofinstability_connect_dset25x2x2x20_inputs.h5')
df25 = df25.reset_index(drop=True)

# Drop pipeline not of interest
df25 = df25[df25["pipeline"] == "prob"]
print(len(df25))
df25.reset_index(drop=True, inplace=True)

# Drop some columns (will re-add relevant ones later)
df25.drop(labels=['bmi', 'pulse', 'age', 'sex', 'seed', 'pipeline'], axis=1, inplace=True)
df25.rename(columns={"directions":"subsample"}, inplace=True)
subs25 = list(set(df25.subject))

2100


In [8]:
df_pheno = pd.read_csv('../data/phenotypic_w_kg.csv')

In [9]:
for cov in ['age', 'sex', 'bmi', 'cholesterol', 'rel_vo2max']:
    df25[cov] = np.nan

for sub in subs25:
    for cov in ['age', 'sex', 'bmi', 'cholesterol', 'rel_vo2max']:
        df25.loc[df25['subject'] == sub, cov] = df_pheno[df_pheno['id'] == sub][cov].values[0]
        
        ses = df25.loc[df25['subject'] == sub, 'session'].values
        sesle = LabelEncoder().fit(ses)
        df25.loc[df25['subject'] == sub, 'session'] = sesle.transform(ses)
        
        samp = df25.loc[df25['subject'] == sub, 'subsample'].values
        sample = LabelEncoder().fit(samp)
        df25.loc[df25['subject'] == sub, 'subsample'] = sample.transform(samp)

subs = list(df25['subject'].unique())
for idx, row in df25.iterrows():
    # Replace subids with unique numbers
    try:
        loc = subs.index(row.subject)
    except ValueError:
        continue
    row['subject'] = loc
    del loc
    
    row['age'] = int(float(row['age']) > 18)
    row['sex'] = int(row['sex'])
    row['bmi'] = int(float(row['bmi']) > 25)
    row['rel_vo2max'] = int(float(row['rel_vo2max']) > (45 + (row['sex']-1)*10))
    row['cholesterol'] = int(float(row['cholesterol']) > 200)
    df25.iloc[idx] = row
    del row

In [10]:
subdf = df25.query('simulation == "ref" and session == 0 and subsample == 0')

print(subdf['cholesterol'].value_counts())
print(subdf['age'].value_counts())
print(subdf['bmi'].value_counts())
print(subdf['rel_vo2max'].value_counts())
print(subdf['sex'].value_counts())

0.0    22
1.0     3
Name: cholesterol, dtype: int64
0.0    15
1.0    10
Name: age, dtype: int64
0.0    13
1.0    12
Name: bmi, dtype: int64
1.0    16
0.0     9
Name: rel_vo2max, dtype: int64
1.0    15
2.0    10
Name: sex, dtype: int64


In [11]:
df.to_hdf('../data/aggregation_connect+demo_dset25x2x2x20.h5', 'data')

In [7]:
subs = list(set(df.subject)) + list(set(df25.subject))

In [8]:
# Load DF
df_pheno = pd.read_csv('../data/phenotypic.csv')
df_pheno2 = pd.read_csv('../data/data-2020-10-23T15_02_42.780Z.csv')

# Deal with minor reorg details
df_pheno["session"] = [_.split(',')[1] for _ in df_pheno['id'].values]
df_pheno["id"] = [_.split(',')[0] for _ in df_pheno['id'].values]
df_pheno2["id"] = [_.split(',')[0] for _ in df_pheno2['id'].values]

keep = inlist(df_pheno['id'], subs)
df_pheno = df_pheno[keep]

keep = inlist(df_pheno2['id'], subs)
df_pheno2 = df_pheno2[keep]

for col in ['sex', 'vo2max', 'bmi', 'cholesterol']:
    tmp = [str(_).split()[0]
           for _ in df_pheno[col].values]
    df_pheno[col] = [np.nan if np.isnan(float(_))
                     else float(_)
                     for _ in tmp]

wgtdf = []
for sub in df_pheno2['id'].unique():
    tmpdf = df_pheno2[df_pheno2['id'] == sub]
    wgt = np.mean(tmpdf['kg'].values)
    wgtdf += [{'id':sub, 'weight(kg)':wgt}]

# Prune for completeness
df_pheno = df_pheno.merge(pd.DataFrame.from_dict(wgtdf), how='left', on='id')
df_pheno["rel_vo2max"] = df_pheno["vo2max"] / df_pheno['weight(kg)'] * 1000
df_pheno.reset_index(drop=True, inplace=True)

In [9]:
df_pheno

,id,age,sex,vo2max,bmi,cholesterol,session,weight(kg),rel_vo2max
0,A00018030,11.673973,2.0,NaN,15.581133,148.0,BAS1,38.555000,NaN
1,A00018030,13.545205,2.0,3.424585,17.918871,140.0,FLU1,38.555000,88.823364
2,A00019903,83.183562,1.0,NaN,22.525657,134.0,BAS1,71.210000,NaN
3,A00028400,49.016438,2.0,2.946664,24.210798,205.0,BAS1,74.400000,39.605699
4,A00028468,46.284932,1.0,4.897387,29.800016,197.0,BAS1,88.900000,55.088716
...,...,...,...,...,...,...,...,...,...
165,A00065379,42.271233,1.0,4.897387,42.039886,NaN,BAS1,134.700000,36.357734
166,A00065574,23.720548,1.0,8.520437,26.181506,138.0,BAS1,85.300000,99.887888
167,A00066396,14.739726,1.0,3.338942,25.180785,149.0,BAS1,82.233333,40.603269
168,A00066396,15.586301,1.0,4.397490,26.046155,188.0,FLU1,82.233333,53.475757


In [10]:
df_pheno.to_csv('../data/phenotypic_w_kg.csv')